In [1]:
# ========================================================================================
# PROBLEM B3
#
# Build a CNN based classifier for Rock-Paper-Scissors dataset.
# Your input layer should accept 150x150 with 3 bytes color as the input shape.
# This is unlabeled data, use ImageDataGenerator to automatically label it.
# Don't use lambda layers in your model.
#
# The dataset used in this problem is created by Laurence Moroney (laurencemoroney.com).
#
# Desired accuracy AND validation_accuracy > 83%
# ========================================================================================

import urllib.request
import zipfile
import tensorflow as tf
import os
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam


def solution_B3():
    data_url = 'https://github.com/dicodingacademy/assets/releases/download/release-rps/rps.zip'
    urllib.request.urlretrieve(data_url, 'rps.zip')
    local_file = 'rps.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/')
    zip_ref.close()

    TRAINING_DIR = "data/rps/"

    training_datagen = ImageDataGenerator (rescale = 1./255,
                                       horizontal_flip = True,
                                       vertical_flip = True,
                                       rotation_range = 20,
                                       width_shift_range = 0.2,
                                       height_shift_range = 0.2,
                                       shear_range = 0.2,
                                       fill_mode="nearest",
                                       validation_split = 0.1,
                                       )
    train_generator = training_datagen.flow_from_directory(TRAINING_DIR,
                                                      batch_size = 32,
                                                      target_size = (150,150),
                                                      class_mode = "categorical",
                                                      subset = "training"
                                                      )

    validation_generator = training_datagen.flow_from_directory(TRAINING_DIR,
                                                         batch_size = 9,
                                                         target_size = (150,150),
                                                         class_mode = "categorical",
                                                         subset = "validation")

    # YOUR IMAGE SIZE SHOULD BE 150x150
    # Make sure you used "categorical"
    class CustomCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
          if (logs.get("accuracy")>0.91):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training = True
    callbacks = CustomCallback()
    model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(16,(3,3),input_shape =(150,150,3),activation = "relu"),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(32,(3,3),activation = "relu"),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(64,(3,3),activation = "relu"),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(64,(3,3),activation = "relu"),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.Dense(512,activation="relu"),                                    
                                    tf.keras.layers.Dense(3,activation = "softmax")
                          ])
    model.summary()
    model.compile(loss  = "mean_squared_error",
              optimizer= Adam(learning_rate = 0.0001),
              metrics = ["accuracy"])

    model.fit(train_generator,
          validation_data  = validation_generator,
          steps_per_epoch = 32,
          validation_steps = 8, 
          epochs = 100,
          callbacks = callbacks)
    return model


# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_B3()
    model.save("model_B3.h5")


Found 2268 images belonging to 3 classes.
Found 252 images belonging to 3 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64)        18496     
                                       